In [1]:
import json
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jsonlines
import altair as alt
from vega_datasets import data



os.chdir("../")

SRC_PATH = ["src"]
for module_path in SRC_PATH:
    if module_path not in sys.path:
        sys.path.append(module_path)

from utils import *

# Experiment 2

## Full finetuning results

In [2]:
dataset_names = ["phi", "gemma", "mistral", "gemma_chat", "zephyr", "llama3", "round_robin"]
training_method = "full_finetuning"
trained_on_models = {"distil_roberta-base": {"03_05_1735": "phi", "03_05_1741": "gemma", "03_05_1748": "mistral", "03_05_1754": "round_robin"},
                    "roberta_large": {"03_05_1845": "phi", "03_05_1910": "gemma", "03_05_1935": "mistral", "03_05_2001": "round_robin"},
                    "electra_large": {"03_05_1842": "phi", "03_05_1909": "gemma", "03_05_1935": "mistral", "03_05_2001": "round_robin"}}


freeze_base_df = create_df_from_test_logs("full_finetuning", trained_on_models, dataset_names)
#heatmap_from_df(freeze_base_df, "accuracy")

In [3]:
freeze_base_df = freeze_base_df.sort_values(by="trained_on_dataset")
dataset_order = ["phi", "gemma", "mistral", "round_robin", "gemma_chat", "zephyr", "llama3"]
freeze_base_df = freeze_base_df.set_index("dataset").loc[dataset_order].reset_index()
detector_name_to_short_name = {"distil_roberta-base": "distil", "roberta_large": "roberta", "electra_large": "electra"}
freeze_base_df["detector_short_name"] = freeze_base_df["base_detector"].apply(lambda x: detector_name_to_short_name[x])
# set detector_name as f"{detector_short_name}_{trained_on_dataset}"
freeze_base_df["detector_name"] = freeze_base_df["detector_short_name"] + "_" + freeze_base_df["trained_on_dataset"]
freeze_base_df.head()

,dataset,accuracy,precision,recall,f1_score,fp_rate,std_accuracy,std_precision,std_recall,std_f1_score,std_fp_rate,TP,TN,FP,FN,base_detector,trained_on_dataset,detector,detector_short_name,detector_name
0,phi,0.947539,0.921121,0.978991,0.949152,0.083968,0.004890,0.008095,0.004428,0.004867,0.008634,975.819,911.679,83.563,20.939,roberta_large,gemma,roberta_large_gemma,roberta,roberta_gemma
1,phi,0.964984,0.957048,0.973710,0.965290,0.043752,0.004102,0.006376,0.005173,0.004190,0.006385,970.558,951.691,43.551,26.200,electra_large,gemma,electra_large_gemma,electra,electra_gemma
2,phi,0.945348,0.942504,0.948649,0.945541,0.057955,0.005259,0.007493,0.007078,0.005370,0.007521,945.572,937.561,57.681,51.186,distil_roberta-base,gemma,distil_roberta-base_gemma,distil,distil_gemma
3,phi,0.949168,0.928045,0.973925,0.950408,0.075626,0.004905,0.008146,0.004834,0.004921,0.008523,970.769,919.974,75.268,25.989,electra_large,mistral,electra_large_mistral,electra,electra_mistral
4,phi,0.955619,0.932033,0.982984,0.956811,0.071784,0.004458,0.007439,0.004227,0.004445,0.007775,979.794,923.799,71.443,16.964,roberta_large,mistral,roberta_large_mistral,roberta,roberta_mistral


In [4]:
freeze_base_df.head(20)

,dataset,accuracy,precision,recall,f1_score,fp_rate,std_accuracy,std_precision,std_recall,std_f1_score,std_fp_rate,TP,TN,FP,FN,base_detector,trained_on_dataset,detector,detector_short_name,detector_name
0,phi,0.947539,0.921121,0.978991,0.949152,0.083968,0.004890,0.008095,0.004428,0.004867,0.008634,975.819,911.679,83.563,20.939,roberta_large,gemma,roberta_large_gemma,roberta,roberta_gemma
1,phi,0.964984,0.957048,0.973710,0.965290,0.043752,0.004102,0.006376,0.005173,0.004190,0.006385,970.558,951.691,43.551,26.200,electra_large,gemma,electra_large_gemma,electra,electra_gemma
2,phi,0.945348,0.942504,0.948649,0.945541,0.057955,0.005259,0.007493,0.007078,0.005370,0.007521,945.572,937.561,57.681,51.186,distil_roberta-base,gemma,distil_roberta-base_gemma,distil,distil_gemma
3,phi,0.949168,0.928045,0.973925,0.950408,0.075626,0.004905,0.008146,0.004834,0.004921,0.008523,970.769,919.974,75.268,25.989,electra_large,mistral,electra_large_mistral,electra,electra_mistral
4,phi,0.955619,0.932033,0.982984,0.956811,0.071784,0.004458,0.007439,0.004227,0.004445,0.007775,979.794,923.799,71.443,16.964,roberta_large,mistral,roberta_large_mistral,roberta,roberta_mistral
5,phi,0.957894,0.949887,0.966854,0.958275,0.051074,0.004367,0.006684,0.005739,0.004449,0.006730,963.719,944.406,50.836,33.039,distil_roberta-base,mistral,distil_roberta-base_mistral,distil,distil_mistral
6,phi,0.966063,0.940545,0.995082,0.967030,0.063007,0.003887,0.007039,0.002170,0.003831,0.007519,991.857,932.541,62.701,4.901,roberta_large,phi,roberta_large_phi,roberta,roberta_phi
7,phi,0.981562,0.969922,0.993972,0.981791,0.030866,0.002999,0.005423,0.002324,0.002999,0.005537,990.751,964.520,30.722,6.007,electra_large,phi,electra_large_phi,electra,electra_phi
8,phi,0.967521,0.951754,0.985017,0.968085,0.050003,0.004080,0.006826,0.003688,0.004088,0.007035,981.827,945.474,49.768,14.931,distil_roberta-base,phi,distil_roberta-base_phi,distil,distil_phi
9,phi,0.953032,0.929037,0.981073,0.954327,0.075061,0.004568,0.007543,0.004232,0.004533,0.008043,977.893,920.547,74.695,18.865,roberta_large,round_robin,roberta_large_round_robin,roberta,roberta_round_robin


### No cross model

In [5]:
# only keep entries where trained_on_dataset == dataset
no_cross_model_df = freeze_base_df[freeze_base_df["trained_on_dataset"] == freeze_base_df["dataset"]]

heatmap = alt.Chart(no_cross_model_df).mark_rect().encode(
    alt.Y('dataset:N', sort=None),
    #alt.Y('detector_short_name:N', sort=None, title="Detector"),
    alt.Color('accuracy:Q').scale(scheme='redyellowgreen', domain=(0.90, 1)),
    #alt.Row("trained_on_dataset:N", title="Dataset used for training"),
).properties(
    width=100,
    height=200
)

heatmap_text = alt.Chart(no_cross_model_df).mark_text(baseline='middle').encode(
    #alt.X('dataset:N', sort=None, title="Dataset used for training and testing"),
    alt.Y('dataset:N', sort=None),
    #alt.Y('detector_short_name:N', sort=None, title="Detector"),
    text='accuracy:Q',
    color=alt.condition(
        alt.datum.accuracy > 0.5,
        alt.value('black'),
        alt.value('white')
    )
).properties(
    width=100,
    height=200
)

chart = alt.layer(heatmap, heatmap_text).facet(
    column=alt.Column("detector_short_name:N", title="Detector")
).configure(
    numberFormat='0.2f'
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)
chart.save("notebooks/plots/heatmap_no_cross_llm.png")
chart

alt.FacetChart(...)

In [22]:
# same as above but with bar chart
bar_chart = alt.Chart(no_cross_model_df).mark_bar().encode(
    #alt.X('detector_short_name:N', sort=None, title="Detector"),
    alt.X('detector_short_name:N', sort=None, title=None),
    alt.Y('accuracy:Q').scale(alt.Scale(domain=(0.85, 1), clamp=True)),
    #alt.Y('accuracy:Q'),
    alt.Color("detector_short_name:N", title="Detector"),
    column=alt.Column("dataset:N", title="Train and Test dataset")
).properties(
    width=100,
    height=200,  
).configure_axis(
    labelFontSize=18,
    titleFontSize=18,
).configure_legend(
    labelFontSize=18,
    titleFontSize=18,
    titleLimit=0
).configure_header(
    titleFontSize=18,
    labelFontSize=18
)
bar_chart.save("notebooks/plots/heatmap_no_cross_llm_bar.png")
bar_chart

alt.Chart(...)

### Cross model

In [29]:
chat_models = ["gemma_chat", "zephyr", "llama3"]
no_chat_df = freeze_base_df[freeze_base_df["dataset"].apply(lambda x: x not in chat_models)]
# select electra only
no_chat_df = no_chat_df[no_chat_df["detector_short_name"] == "electra"]

heatmap = alt.Chart(no_chat_df).mark_rect().encode(
    alt.X('dataset:N', sort="x", title="Tested on"),
    alt.Y('trained_on_dataset:N', sort="y", title="Trained on"),
    alt.Color('accuracy:Q').scale(scheme='redyellowgreen', domain=[0.85, 1]),
    #alt.Row("trained_on_dataset:N", title="Dataset used for training"),
).properties(
    width=300,
    height=300
)

heatmap_text = alt.Chart(no_chat_df).mark_text(baseline='middle').encode(
    alt.X('dataset:N', sort="x", title="Tested on"),
    alt.Y('trained_on_dataset:N', sort="y", title="Trained on"),
    text='accuracy:Q',
    color=alt.condition(
        alt.datum.accuracy > 0.5,
        alt.value('black'),
        alt.value('white')
    )
).properties(
    width=300,
    height=300
)

heatmap_cross = alt.layer(heatmap, heatmap_text).facet(
    #column=alt.Column("trained_on_dataset:N", title="Dataset used for training")
    column=alt.Column("detector_short_name:N", title="Detector")
).configure(
    numberFormat='0.2f'
).configure_axis(
    labelFontSize=18,
    titleFontSize=18
).configure_legend(
    labelFontSize=18,
    titleFontSize=18,
    titleLimit=0
).configure_header(
    titleFontSize=18,
    labelFontSize=18
)

heatmap_cross.save("notebooks/plots/heatmap_cross_llm.png")
heatmap_cross

alt.FacetChart(...)

### Chat Models only

In [34]:
chat_models = ["gemma_chat", "zephyr", "llama3"]
chat_only_df = freeze_base_df[freeze_base_df["dataset"].apply(lambda x: x in chat_models)]
#chat_only_phi_df = chat_only_df[chat_only_df["trained_on_dataset"] == "phi"]
chat_only_electra_df = chat_only_df[chat_only_df["detector_short_name"] == "electra"]

heatmap = alt.Chart(chat_only_electra_df).mark_rect().encode(
    alt.X('dataset:N', sort="x", title="Tested on"),
    alt.Y('trained_on_dataset:N', sort=None, title="Trained on"),
    alt.Color('accuracy:Q').scale(scheme='redyellowgreen', domain=[0.85, 1]),
    #alt.Row("trained_on_dataset:N", title="Dataset used for training"),
).properties(
    width=300,
    height=300
)

heatmap_text = alt.Chart(chat_only_electra_df).mark_text(baseline='middle').encode(
    alt.X('dataset:N', sort="x", title="Tested on"),
    alt.Y('trained_on_dataset:N', sort=None, title="Trained on"),
    text='accuracy:Q',
    color=alt.condition(
        alt.datum.accuracy > 0.5,
        alt.value('black'),
        alt.value('white')
    )
).properties(
    width=300,
    height=300
)

chart = alt.layer(heatmap, heatmap_text).facet(
    column=alt.Column("detector_short_name:N", title="Detector")
).configure(
    numberFormat='0.2f'
).configure_axis(
    labelFontSize=18,
    titleFontSize=18
).configure_legend(
    labelFontSize=18,
    titleFontSize=18,
    titleLimit=0
).configure_header(
    titleFontSize=18,
    labelFontSize=18
)

chart.save("notebooks/plots/heatmap_chat_only.png")
chart

alt.FacetChart(...)

### Zero shot results

In [35]:
fast_detect_gpt_results = {"fast_detect_gpt": {"07_05_0942" : "phi", "07_05_0949" : "gemma", "07_05_0956" : "mistral", "07_05_1003" : "round_robin",
                           "07_05_1007": "gemma_chat", "07_05_1014" : "zephyr", "07_05_1020" : "llama3"}}
roberta_open_ai_results = {"roberta_base_open_ai": {"06_05_1716" : "phi", "06_05_1718" : "gemma", "06_05_1719" : "mistral", "06_05_1721" : "round_robin",
                           "06_05_1723": "gemma_chat", "06_05_1724" : "zephyr", "06_05_1726" : "llama3"}}
freeze_base_df = create_df_from_test_logs("full_finetuning", trained_on_models, dataset_names)

freeze_base_df = add_test_logs_to_results_df(freeze_base_df, fast_detect_gpt_results)
freeze_base_df = add_test_logs_to_results_df(freeze_base_df, roberta_open_ai_results)

# create detector_short_name column for the new detectors
detector_name_to_short_name = {"distil_roberta-base": "distil", "roberta_large": "roberta", "electra_large": "electra", "fast_detect_gpt": "fast_detect_gpt", "roberta_base_open_ai": "roberta_open_ai"}
freeze_base_df["detector_short_name"] = freeze_base_df["base_detector"].apply(lambda x: detector_name_to_short_name[x])

In [36]:
zero_shot_detectors = ["fast_detect_gpt", "roberta_base_open_ai"]
zero_shot_only_only_df = freeze_base_df[freeze_base_df["detector"].apply(lambda x: x in zero_shot_detectors)]

heatmap = alt.Chart(zero_shot_only_only_df).mark_rect().encode(
    alt.X('dataset:N', sort="x", title="Testing on"),
    alt.Y('detector_short_name:N', sort=None, title="Detector"),
    alt.Color('accuracy:Q').scale(scheme='redyellowgreen'),
    #alt.Row("trained_on_dataset:N", title="Dataset used for training"),
).properties(
    width=200,
    height=200
)

heatmap_text = alt.Chart(zero_shot_only_only_df).mark_text(baseline='middle').encode(
    alt.X('dataset:N', sort="x", title="Testing on"),
    alt.Y('detector_short_name:N', sort=None, title="Detector"),
    text='accuracy:Q',
    color=alt.condition(
        alt.datum.accuracy > 0.5,
        alt.value('black'),
        alt.value('white')
    )
).properties(
    width=200,
    height=200
)

chart = alt.layer(heatmap, heatmap_text).configure(
    numberFormat='0.2f'
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)

chart.save("notebooks/plots/heatmap_zero_shot.png")
chart

alt.LayerChart(...)

In [38]:
zero_shot_detectors = ["fast_detect_gpt", "roberta_base_open_ai"]
zero_shot_only_only_df = freeze_base_df[freeze_base_df["detector"].apply(lambda x: x in zero_shot_detectors)]

heatmap = alt.Chart(zero_shot_only_only_df).mark_rect().encode(
    alt.X('dataset:N', sort="x", title="Testing on"),
    #alt.Y('detector_short_name:N', sort=None, title="Detector"),
    alt.Color('accuracy:Q').scale(scheme='redyellowgreen'),
    #alt.Row("trained_on_dataset:N", title="Dataset used for training"),
).properties(
    width=200,
    height=100
)

heatmap_text = alt.Chart(zero_shot_only_only_df).mark_text(baseline='middle').encode(
    alt.X('dataset:N', sort="x", title="Testing on"),
    #alt.Y('detector_short_name:N', sort=None, title="Detector"),
    text='accuracy:Q',
    color=alt.condition(
        alt.datum.accuracy > 0.5,
        alt.value('black'),
        alt.value('white')
    )
).properties(
    width=200,
    height=100
)

chart = alt.layer(heatmap, heatmap_text).facet(
    column=alt.Column("detector_short_name:N", title="Detector")
).configure(
    numberFormat='0.2f'
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)

chart.save("notebooks/plots/heatmap_zero_shot.png")
chart

alt.FacetChart(...)

In [87]:
zero_shot_detectors_and_best_trained = ["fast_detect_gpt", "roberta_base_open_ai", "electra_large_phi"]
zero_shot_only_only_df = freeze_base_df[freeze_base_df["detector"].apply(lambda x: x in zero_shot_detectors_and_best_trained)]

# only keep chat datasets
chat_models = ["gemma_chat", "zephyr", "llama3"]
zero_shot_only_only_df = zero_shot_only_only_df[zero_shot_only_only_df["dataset"].apply(lambda x: x in chat_models)]


# same as above but with bar chart
bar_chart = alt.Chart(zero_shot_only_only_df).mark_bar().encode(
    alt.X('detector_short_name:N', sort=None, title=None),
    alt.Y('accuracy:Q').scale(alt.Scale(domain=(0.6, 1), clamp=True)),
    #alt.Y('accuracy:Q'),
    alt.Color("detector_short_name:N", title="Detector"),
    column=alt.Column("dataset:N", title="Test dataset")
).properties(
    width=100,
    height=200,  
).configure_axis(
    labelFontSize=18,
    titleFontSize=18,
).configure_legend(
    labelFontSize=18,
    titleFontSize=18,
    titleLimit=0
).configure_header(
    titleFontSize=18,
    labelFontSize=18
)
bar_chart.save("notebooks/plots/heatmap_zero_shot_bar.png")
bar_chart

alt.Chart(...)

In [78]:
zero_shot_detectors_and_best_trained = ["fast_detect_gpt", "roberta_base_open_ai", "electra_large_phi"]
zero_shot_only_only_df = freeze_base_df[freeze_base_df["detector"].apply(lambda x: x in zero_shot_detectors_and_best_trained)]

# same as above but with bar chart
bar_chart = alt.Chart(zero_shot_only_only_df).mark_bar().encode(
    alt.X('detector_short_name:N', sort=None, title=None),
    alt.Y('accuracy:Q').scale(alt.Scale(domain=(0.6, 1), clamp=True)),
    #alt.Y('accuracy:Q'),
    alt.Color("detector_short_name:N", title="Detector"),
    column=alt.Column("dataset:N", title=None)
).properties(
    width=100,
    height=100,  
)

# spread the char into two rows
grid_chart = alt.vconcat()
nb_rows = 2
nb_dataset_per_row = 4

datasets = zero_shot_only_only_df["dataset"].unique()
for i in range(nb_rows):
    row = alt.hconcat()
    for j in range(nb_dataset_per_row):
        if i*nb_dataset_per_row + j >= len(datasets):
            break
        row |= bar_chart.transform_filter(
            alt.datum.dataset == datasets[i*nb_dataset_per_row + j]
        )
    grid_chart &= row

grid_chart.configure_axis(
    labelFontSize=12,
    titleFontSize=12,
).configure_legend(
    labelFontSize=12,
    titleFontSize=12,
    titleLimit=0
).configure_header(
    titleFontSize=12,
    labelFontSize=12
)
grid_chart.save("notebooks/plots/heatmap_zero_shot_bar.png")

grid_chart

alt.VConcatChart(...)

In [79]:
zero_shot_detectors_and_best_trained = ["fast_detect_gpt", "roberta_base_open_ai", "electra_large_phi"]
zero_shot_only_only_df = freeze_base_df[freeze_base_df["detector"].apply(lambda x: x in zero_shot_detectors_and_best_trained)]

# same as above but with bar chart
bar_chart = alt.Chart(zero_shot_only_only_df).mark_bar().encode(
    alt.X('detector_short_name:N', sort=None, title=None),
    alt.Y('accuracy:Q').scale(alt.Scale(domain=(0.6, 1), clamp=True)),
    #alt.Y('accuracy:Q'),
    alt.Color("detector_short_name:N", title="Detector"),
    column=alt.Column("dataset:N", title=None)
).properties(
    width=100,
    height=100,  
)

# spread the char into two rows
grid_chart = alt.vconcat()
nb_rows = 1
nb_dataset_per_row = 3

#datasets = zero_shot_only_only_df["dataset"].unique()
datasets = ["gemma_chat", "zephyr", "llama3"]
for i in range(nb_rows):
    row = alt.hconcat()
    for j in range(nb_dataset_per_row):
        if i*nb_dataset_per_row + j >= len(datasets):
            break
        row |= bar_chart.transform_filter(
            alt.datum.dataset == datasets[i*nb_dataset_per_row + j]
        )
    grid_chart &= row

grid_chart.configure_axis(
    labelFontSize=18,
    titleFontSize=18,
).configure_legend(
    labelFontSize=18,
    titleFontSize=18,
    titleLimit=0
).configure_header(
    titleFontSize=18,
    labelFontSize=18
)
grid_chart.save("notebooks/plots/heatmap_zero_shot_bar.png")

grid_chart

alt.VConcatChart(...)

In [63]:
datasets

array(['gemma', 'gemma_chat', 'llama3', 'mistral', 'phi', 'round_robin',
       'zephyr'], dtype=object)

In [23]:
# best detector

# change type of accuracy, precision, recall, f1-score to float
freeze_base_df["accuracy"] = freeze_base_df["accuracy"].astype(float)
freeze_base_df["precision"] = freeze_base_df["precision"].astype(float)
freeze_base_df["recall"] = freeze_base_df["recall"].astype(float)
freeze_base_df["f1_score"] = freeze_base_df["f1_score"].astype(float)

freeze_base_df_metrics = freeze_base_df[["accuracy", "precision", "recall", "f1_score", "detector"]]

# group by detector and compute the mean accuracy per detector
freeze_base_df_metrics.groupby(["detector"]).mean().reset_index().sort_values(by="accuracy", ascending=False)

,detector,accuracy,precision,recall,f1_score
6,electra_large_phi,0.961782,0.973659,0.949037,0.960694
4,electra_large_gemma,0.961628,0.963032,0.960107,0.961369
7,electra_large_round_robin,0.957071,0.950538,0.964544,0.957243
10,roberta_large_phi,0.954543,0.937130,0.973597,0.954834
5,electra_large_mistral,0.947887,0.924454,0.974840,0.948762
11,roberta_large_round_robin,0.943282,0.919024,0.972170,0.944423
8,roberta_large_gemma,0.941932,0.916227,0.972282,0.943109
9,roberta_large_mistral,0.941863,0.915510,0.974700,0.943407
2,distil_roberta-base_phi,0.941789,0.956639,0.925422,0.940294
1,distil_roberta-base_mistral,0.941352,0.955859,0.925363,0.939687


In [24]:
# base detector

freeze_base_df_metrics = freeze_base_df[["accuracy", "precision", "recall", "f1_score", "base_detector"]]

# group by detector and compute the mean accuracy per detector
freeze_base_df_metrics.groupby(["base_detector"]).mean().reset_index().sort_values(by="accuracy", ascending=False)

,base_detector,accuracy,precision,recall,f1_score
1,electra_large,0.957092,0.952921,0.962132,0.957017
2,roberta_large,0.945405,0.921973,0.973187,0.946443
0,distil_roberta-base,0.936492,0.942474,0.930817,0.935815


In [27]:
# best dataset to train on

freeze_base_df_metrics= freeze_base_df[["accuracy", "precision", "recall", "f1_score", "trained_on_dataset"]].copy()
freeze_base_df_metrics.groupby(["trained_on_dataset"]).mean().reset_index().sort_values(by="accuracy", ascending=False)

,trained_on_dataset,accuracy,precision,recall,f1_score
2,phi,0.952705,0.955809,0.949352,0.951940
0,gemma,0.946979,0.943034,0.951946,0.946860
1,mistral,0.943701,0.931941,0.958301,0.943952
3,round_robin,0.941935,0.925705,0.961915,0.942947


In [21]:
# best detectors on chat models
chat_models = ["gemma_chat", "zephyr", "llama3"]
freeze_base_df_metrics_chat = freeze_base_df[["accuracy", "precision", "recall", "f1_score", "detector", "dataset"]].copy()
freeze_base_df_metrics_chat = freeze_base_df_metrics_chat[freeze_base_df_metrics_chat["dataset"].isin(chat_models)]
freeze_base_df_metrics_chat = freeze_base_df_metrics_chat[["accuracy", "precision", "recall", "f1_score", "detector"]]
freeze_base_df_metrics_chat.groupby(["detector"]).mean().reset_index().sort_values(by="accuracy", ascending=False)

,detector,accuracy,precision,recall,f1_score
10,roberta_large_phi,0.960365,0.941962,0.981262,0.961156
6,electra_large_phi,0.960018,0.970159,0.949317,0.959208
7,electra_large_round_robin,0.950937,0.943454,0.959467,0.951116
11,roberta_large_round_robin,0.949373,0.931687,0.969955,0.950355
4,electra_large_gemma,0.949170,0.957301,0.940354,0.948654
5,electra_large_mistral,0.948881,0.930841,0.969896,0.949890
9,roberta_large_mistral,0.947042,0.933182,0.963109,0.947779
8,roberta_large_gemma,0.943025,0.925633,0.963602,0.944131
2,distil_roberta-base_phi,0.935869,0.950208,0.920031,0.934594
1,distil_roberta-base_mistral,0.921012,0.947841,0.891160,0.918248


In [28]:
# dataset where detectors struggle the most

freeze_base_df_metrics = freeze_base_df[["accuracy", "precision", "recall", "f1_score", "dataset"]].copy()
freeze_base_df_metrics.groupby(["dataset"]).mean().reset_index().sort_values(by="accuracy", ascending=True)

,dataset,accuracy,precision,recall,f1_score
6,zephyr,0.931474,0.942404,0.919633,0.930277
1,gemma_chat,0.941336,0.936464,0.947515,0.941615
3,mistral,0.943558,0.935949,0.952718,0.943913
5,round_robin,0.947610,0.945933,0.951881,0.946317
2,llama3,0.951750,0.937745,0.968335,0.952491
0,gemma,0.952164,0.937208,0.969822,0.952960
4,phi,0.956415,0.938155,0.977747,0.957402
